In [1]:
import os

import petsc4py
import underworld3 as uw
from underworld3 import timing

import numpy as np
import sympy
import argparse
import pickle

import os

import petsc4py
import underworld3 as uw
from underworld3 import timing

import numpy as np
import sympy
import argparse
import pickle

# parser = argparse.ArgumentParser()
# parser.add_argument('-i', "--idx", type=int, required=True)
# parser.add_argument('-p', "--prev", type=int, required=True) # set to 0 if no prev_res, 1 if there is
# parser.add_argument('-t', "--dt", type=float, required=True) # deltaT
# parser.add_argument('-s', "--ms", type=int, required=True) # maxsteps
# args = parser.parse_args()

# idx = args.idx
# prev = args.prev
# dt_ns = args.dt
# maxsteps = args.ms

idx = 0
prev = 0
dt_ns = 0.5
maxsteps = 1

mesh_use = str(os.getenv("MESH_USE", "struct_quad"))
if uw.mpi.rank == 0:
    print(f"Mesh used: {mesh_use}")
res = int(os.getenv("RES", 16))
print(f"Resolution: {res}")

#if dt_ns == 0.1:
#    maxsteps = 5 # 5
#else:
#    maxsteps = 50 # 10

velocity = 1

tol = 1e-5
adv_type = "vector" # scalar or vector
vel_type = "v_rigid_body" # v_irrotational or v_rigid_body
#mesh_use = "struct_quad" # struct_quad or simp_irreg or simp_reg

qdeg     = 3
Vdeg     = 2
sl_order = 1

if uw.mpi.rank == 0:
    print(maxsteps*dt_ns*velocity)

Mesh used: struct_quad
Resolution: 16
0.5


In [2]:
outdir = "/Users/jgra0019/Documents/codes/uw3-dev/Navier-Stokes-benchmark/plots-SLCN-test"

#outfile = f"VecAdv-run{idx}"
outfile = f"VecAdv-run" # overwrite outputted files to reduce total size
outdir = f"./VecAdv-{mesh_use}-{vel_type}-res{res}-order{sl_order}-dt{dt_ns}"

if prev == 0:
    prev_idx = 0
    infile = None
else:
    prev_idx = int(idx) - 1
    #infile = f"VecAdv-run{prev_idx}"
    infile = f"VecAdv-run"

if uw.mpi.rank == 0:
    os.makedirs(outdir, exist_ok=True)

In [3]:
# ### mesh coordinates
xmin, xmax = 0., 2.
ymin, ymax = 0., 1.

sdev = 0.1
x0 = 0.5
y0 = 0.5

# ### Set up the mesh
### Quads
if mesh_use == "struct_quad":
    mesh = uw.meshing.StructuredQuadBox(
        elementRes=(int(res*xmax), int(res)),
        minCoords=(xmin, ymin),
        maxCoords=(xmax, ymax),
        qdegree=3,
    )
elif mesh_use == "simp_irreg":
    mesh = uw.meshing.UnstructuredSimplexBox(
        minCoords=(xmin,ymin),
        maxCoords=(xmax,ymax),
        cellSize=1 / res, regular=False, qdegree=3, refinement=0
    )
elif mesh_use == "simp_reg":
    mesh = uw.meshing.UnstructuredSimplexBox(
        minCoords=(xmin,ymin),
        maxCoords=(xmax,ymax),
        cellSize=1 / res, regular=True, qdegree=3, refinement=0
    )

In [4]:
import petsc4py.PETSc as PETSc



In [6]:
mesh.dm.get

In [7]:
out = mesh.dm.getCellCoordinatesLocal()
out.array

array([], dtype=float64)

In [29]:
v           = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=Vdeg)

# initial vector field
vec_ini   = uw.discretisation.MeshVariable("Vi", mesh, mesh.dim, degree=Vdeg)

# vector being advected
vec_tst   = uw.discretisation.MeshVariable("Vn", mesh, mesh.dim, degree=Vdeg)
vec_ana   = uw.discretisation.MeshVariable("Va", mesh, mesh.dim, degree=Vdeg)

# vorticity
omega_tst  = uw.discretisation.MeshVariable(r"\omega_t", mesh, 1, degree=2)
omega_ana  = uw.discretisation.MeshVariable(r"\omega_a", mesh, 1, degree=2)

In [5]:
# functions for calculating the norms
# NOTE: these modify the vector and vorticity fields
# so should be done at the end of everything
import math

def calculate_vel_omega_rel_norm():

    # define domain where we perform the integral
    dist_travel = velocity*dt_ns*(idx + 1)*maxsteps
    min_dom = 0.1 * x0 + dist_travel
    max_dom = x0 + dist_travel + 0.9 * x0

    x,y = mesh.X

    mask_fn = sympy.Piecewise((1, (x > min_dom) &  (x < max_dom)), (0., True))

    # sympy functions corresponding to integrals
    vec_diff = vec_tst.sym - vec_ana.sym
    vec_diff_mag = vec_diff.dot(vec_diff)

    vec_ana_mag = vec_ana.sym.dot(vec_ana.sym)

    omega_diff = (omega_tst.sym - omega_ana.sym)**2
    omega_ana_sq = omega_ana.sym**2

    vec_diff_mag_integ = math.sqrt(uw.maths.Integral(mesh, mask_fn * vec_diff_mag).evaluate())
    vec_ana_mag_integ = math.sqrt(uw.maths.Integral(mesh, mask_fn * vec_ana_mag).evaluate())
    vec_norm = vec_diff_mag_integ / vec_ana_mag_integ

    omega_diff_integ = math.sqrt(uw.maths.Integral(mesh, mask_fn * omega_diff).evaluate())
    omega_ana_sq_integ = math.sqrt(uw.maths.Integral(mesh, mask_fn * omega_ana_sq).evaluate())
    omega_norm = omega_diff_integ / omega_ana_sq_integ

    return omega_norm, vec_norm

# def calculate_vel_omega_rms(offset = 0):
#     # NOTE: function not used
#     # define domain where we perform the integral
#     min_dom = 0.1 * x0 + offset
#     max_dom = x0 + offset + 0.9 * x0

#     #print(f"min: {min_dom}, max: {max_dom}")

#     x,y = mesh.X

#     mask_fn = sympy.Piecewise((1, (x > min_dom) &  (x < max_dom)), (0., True))

#     # sympy functions corresponding to integrals
#     vec_tst_mag     = vec_tst.sym.dot(vec_tst.sym)
#     omega_tst_sq    = omega_tst.sym**2
#     area            = uw.maths.Integral(mesh, mask_fn * 1.0).evaluate()

#     vec_tst_rms     = math.sqrt(uw.maths.Integral(mesh, mask_fn * vec_tst_mag).evaluate() / area)
#     omega_tst_rms   = math.sqrt(uw.maths.Integral(mesh, mask_fn * omega_tst_sq).evaluate() / area)

#     return omega_tst_rms, vec_tst_rms

In [6]:
# #### Create the SL object
DuDt = uw.systems.ddt.SemiLagrangian(
                                        mesh,
                                        vec_tst.sym,
                                        v.sym,
                                        vtype = uw.VarType.VECTOR,
                                        degree = Vdeg,
                                        continuous = vec_tst.continuous,
                                        varsymbol = vec_tst.symbol,
                                        verbose = False,
                                        bcs = None,
                                        order = sl_order,
                                        smoothing = 0.0,
                                    )

In [19]:
help(uw.systems.ddt.SemiLagrangian)

Help on class SemiLagrangian in module underworld3.systems.ddt:

class SemiLagrangian(underworld3.utilities._api_tools.uw_object)
 |  SemiLagrangian(mesh: underworld3.discretisation.Mesh, psi_fn: sympy.core.function.Function, V_fn: sympy.core.function.Function, vtype: underworld3._var_types.VarType, degree: int, continuous: bool, varsymbol: Optional[str] = 'u', verbose: Optional[bool] = False, bcs=[], order=1, smoothing=0.0, under_relaxation=0.0, bc_mask_fn=1)
 |  
 |  Nodal-Swarm  Lagrangian History Manager:
 |  This manages the update of a Lagrangian variable, $\psi$ on the swarm across timesteps.
 |  $$\quad \psi_p^{t-n\Delta t} \leftarrow \psi_p^{t-(n-1)\Delta t}\quad$$
 |  $$\quad \psi_p^{t-(n-1)\Delta t} \leftarrow \psi_p^{t-(n-2)\Delta t} \cdots\quad$$
 |  $$\quad \psi_p^{t-\Delta t} \leftarrow \psi_p^{t}$$
 |  
 |  Method resolution order:
 |      SemiLagrangian
 |      underworld3.utilities._api_tools.uw_object
 |      underworld3.utilities._api_tools.uw_object_counter
 |     

In [7]:
# mesh.return_coords_to_bounds??

In [8]:
# ### Set up:
# - Velocity field
# - Initial vector distribution

with mesh.access(v):
    v.data[:, 0] = velocity

x,y = mesh.X

## Irrotational vortex
def v_irrotational(alpha, x0, y0, coords):
    '''
    Irrotational vortex

    $$ (vx, vy) = (-\alpha y r^{-2}, \alpha x r^{-2} $$
    '''

    ar2 = alpha / ((x - x0)**2 + (y - y0)**2 + 0.001)
    return uw.function.evaluate(sympy.Matrix([-ar2 * (y-y0), ar2 * (x-x0)]) ,coords)


def v_rigid_body(alpha, x0, y0, coords):
    '''
    Rigid body vortex (with Gaussian envelope)

    $$ (vx, vy) = (-\Omega y, \Omega y) $$
    '''
    ar2 = sympy.exp(-alpha*((x - x0)**2 + (y - y0)**2 + 0.000001))
    return uw.function.evaluate(sympy.Matrix([-ar2 * (y-y0), ar2 * (x-x0)]) ,coords)

if infile is None:
    with mesh.access(vec_tst, vec_ini):
        if vel_type == "v_irrotational":
            vec_tst.data[:, :] =  v_irrotational(0.01, x0, y0, vec_tst.coords)
            vec_ini.data[:, :] =  v_irrotational(0.01, x0, y0, vec_ini.coords)
        elif vel_type == "v_rigid_body":
            vec_tst.data[:, :] = v_rigid_body(33, x0, y0, vec_tst.coords)
            vec_ini.data[:, :] =  v_rigid_body(33, x0, y0, vec_ini.coords)
else:
    vec_tst.read_timestep(data_filename = infile, data_name = "Vn", index = maxsteps, outputPath = outdir)
    # don't need to read vec_ini

vorticity_calc_test = uw.systems.Projection(mesh, omega_tst)
vorticity_calc_test.uw_function = mesh.vector.curl(vec_tst.sym)
vorticity_calc_test.petsc_options["snes_monitor"]= None
vorticity_calc_test.petsc_options["ksp_monitor"] = None

vorticity_calc_ana = uw.systems.Projection(mesh, omega_ana)
vorticity_calc_ana.uw_function = mesh.vector.curl(vec_ana.sym)
vorticity_calc_ana.petsc_options["snes_monitor"]= None
vorticity_calc_ana.petsc_options["ksp_monitor"] = None

#vorticity_calc_test.solve()

In [9]:
# initial velocity and omega RMS
# of vec_tst - nothing done on vec_ana
#om_rms_init, v_rms_init = calculate_vel_omega_rms(offset = 0)
#if uw.mpi.rank == 0:
#    print(f"Omega RMS: {om_rms_init:.8f}")
#    print(f"V RMS: {v_rms_init:.8f}")

In [10]:
ts = 0
elapsed_time = 0.0
model_time  = 0.0

In [11]:
dt_physical = None
order = 1
evalf = False
verbose = False
dt = dt_ns

In [12]:
DuDt.mesh.return_coords_to_bounds??

Signature: DuDt.mesh.return_coords_to_bounds(coords)
Docstring: <no docstring>
Source:   
    def box_return_coords_to_bounds(coords):

        x00s = coords[:, 0] < minCoords[0]
        x01s = coords[:, 0] > maxCoords[0]
        x10s = coords[:, 1] < minCoords[1]
        x11s = coords[:, 1] > maxCoords[1]

        if dim == 3:
            x20s = coords[:, 1] < minCoords[2]
            x21s = coords[:, 1] > maxCoords[2]

        coords[x00s, :] = minCoords[0]
        coords[x01s, :] = maxCoords[0]
        coords[x10s, :] = minCoords[1]
        coords[x11s, :] = maxCoords[1]

        if dim == 3:
            coords[x20s, :] = minCoords[2]
            coords[x21s, :] = maxCoords[2]

        return coords
File:      ~/Documents/codes/uw3-dev/uw3-local-installation/uw3_dev/src/underworld3/meshing.py
Type:      function

In [13]:
from copy import deepcopy

for step in range(0, maxsteps):

   delta_t = dt_ns
   #DuDt.update_pre_solve(delta_t, verbose = False, evalf = False)

    # re-implement the pre-solve in here
    # change the 
    ## Progress from the oldest part of the history
    # 1. Copy the stored values down the chain

   if dt_physical is not None:
      phi = min(1, dt / dt_physical)
   else:
      phi = sympy.sympify(1)

   for i in range(DuDt.order - 1, 0, -1):
      with DuDt.mesh.access(DuDt.psi_star[i]):
         DuDt.psi_star[i].data[...] = (
               phi * DuDt.psi_star[i - 1].data[...]
               + (1 - phi) * DuDt.psi_star[i].data[...]
         )

   # 2. Compute the upstream values

   # We use the u_star variable as a working value here so we have to work backwards
   print("Test .. ")
   for i in range(DuDt.order - 1, -1, -1):
      print("Here: ", i)
      with DuDt._nswarm_psi.access(DuDt._nswarm_psi._X0):
         DuDt._nswarm_psi._X0.data[...] = DuDt._nswarm_psi.data[...]
         
         before_adv = deepcopy(DuDt._nswarm_psi.data[...])
         
      # march nodes backwards along characteristics
      DuDt._nswarm_psi.advection(
         sympy.Matrix([1., 0]),
         -dt,
         order=2,
         corrector=False,
         #restore_points_to_domain_func=DuDt.mesh.return_coords_to_bounds,
         restore_points_to_domain_func = mesh.return_coords_to_bounds,
         evalf=evalf,
      )
      
      break
      print("After break")
        
    #     if i == 0:
    #         # Recalculate u_star from u_fn
    #         DuDt._psi_star_projection_solver.uw_function = DuDt.psi_fn
    #         DuDt._psi_star_projection_solver.solve(verbose=verbose)

    #     if evalf:
    #         with DuDt._nswarm_psi.access(DuDt._nswarm_psi.swarmVariable):
    #             for d in range(DuDt.psi_star[i].shape[1]):
    #                 DuDt._nswarm_psi.swarmVariable.data[:, d] = uw.function.evalf(
    #                     DuDt.psi_star[i].sym[d], DuDt._nswarm_psi.data
    #                 )
    #     else:
    #         with DuDt._nswarm_psi.access(DuDt._nswarm_psi.swarmVariable):
    #             for d in range(DuDt.psi_star[i].shape[1]):
    #                 DuDt._nswarm_psi.swarmVariable.data[:, d] = (
    #                     uw.function.evaluate(
    #                         DuDt.psi_star[i].sym[d], DuDt._nswarm_psi.data
    #                     )
    #                 )

    #     # with self.mesh.access():
    #     #     print("1:", self.psi_star[0].data, flush=True)

    #     # with self._nswarm_psi.access():
    #     #     print("1S:", self._nswarm_psi.swarmVariable.data, flush=True)

    #     # restore coords (will call dm.migrate after context manager releases)
    #     with DuDt._nswarm_psi.access(DuDt._nswarm_psi.particle_coordinates):
    #         DuDt._nswarm_psi.data[...] = DuDt._nswarm_psi._nX0.data[...]

    #     # Now project to the mesh using bc's to obtain u_star

    #     DuDt._psi_star_projection_solver.uw_function = (
    #         DuDt._nswarm_psi.swarmVariable.sym
    #     )

    #     DuDt._psi_star_projection_solver.solve()

    #     # Copy data from the projection operator if required
    #     if i != 0:
    #         with DuDt.mesh.access(DuDt.psi_star[i]):
    #             DuDt.psi_star[i].data[...] = DuDt.psi_star[0].data[...]


    with mesh.access(vec_tst): # update vector field
       vec_tst.data[...] = DuDt.psi_star[0].data[...]

    model_time += delta_t
    if uw.mpi.rank == 0:
       print(f"{step}: Time - {model_time}")

    step += 1



  0 SNES Function norm 9.274784477758e-04
    Residual norms for  solve.
    0 KSP Residual norm 1.402439231119e+00
    1 KSP Residual norm 3.922495635182e-01
    2 KSP Residual norm 1.310986477821e-01
    3 KSP Residual norm 2.270816669231e-02
    4 KSP Residual norm 7.758502840098e-03
    5 KSP Residual norm 1.725987330701e-03
    6 KSP Residual norm 3.313515992156e-04
  1 SNES Function norm 1.191057664859e-07
    Residual norms for  solve.
    0 KSP Residual norm 3.323053289416e-04
    1 KSP Residual norm 1.253770164987e-04
    2 KSP Residual norm 2.953231961148e-05
    3 KSP Residual norm 8.240847704056e-06
    4 KSP Residual norm 3.604388373053e-06
    5 KSP Residual norm 1.152231599375e-06
    6 KSP Residual norm 1.930152483114e-07
  2 SNES Function norm 1.286741406637e-10
  0 SNES Function norm 9.274550049291e-04
    Residual norms for  solve.
    0 KSP Residual norm 1.419776736204e+00
    1 KSP Residual norm 4.184466807099e-01
    2 KSP Residual norm 1.321489477303e-01
    3 KS

In [14]:
# import matplotlib.pyplot as plt

# with DuDt._nswarm_psi.access(DuDt._nswarm_psi):
#     print(before_adv.shape)
#     cond = DuDt._nswarm_psi.data[:, 0] < 0.1
#     #cond = True
#     fig, ax = plt.subplots(dpi = 300)
#     ax.scatter(before_adv[cond, 0], before_adv[cond, 1], s = 10, label = "Before advection", zorder = 1)
#     ax.scatter(DuDt._nswarm_psi.data[cond, 0], DuDt._nswarm_psi.data[cond, 1], s = 1, label = "After advection", zorder = 2)

#     # ax.scatter(before_adv[:, 0], before_adv[:, 1], s = 10, label = "Before advection", zorder = 1)
#     # ax.scatter(DuDt._nswarm_psi.data[:, 0], DuDt._nswarm_psi.data[:, 1], s = 1, label = "After advection", zorder = 2)
#     #ax.set_aspect("equal")
#     ax.legend()

#     #print(before_adv[cond])

#     #print(DuDt._nswarm_psi.data[cond])
    

In [15]:
# calculate analytical velocity field
dist_travel = velocity*dt_ns*(idx + 1)*maxsteps # travel since first iteration loop
with mesh.access(vec_ana):
    if vel_type == "v_irrotational":
        vec_ana.data[:, :] = v_irrotational(0.01, x0 + dist_travel, y0, vec_ana.coords)
    elif vel_type == "v_rigid_body":
        vec_ana.data[:, :] = v_rigid_body(33, x0 + dist_travel, y0, vec_ana.coords)

# calculate vorticity
vorticity_calc_ana.solve()
vorticity_calc_test.solve()

# current omega and v rms of v_test
#om_rms, v_rms = calculate_vel_omega_rms(offset = velocity*dt_ns*step)
# print(f"Omega RMS: {om_rms:.8f}")
# print(f"V RMS: {v_rms:.8f}")

# courant_num = max(a_param, b_param) * meshbox.get_min_radius()/dt_ns
courant_num = velocity * dt_ns / mesh.get_min_radius()
om_norm, v_norm = calculate_vel_omega_rel_norm()

# Relative difference in omega_rms and v_rms
#om_rms_rel = abs(om_rms_init - om_rms)/om_rms_init
#v_rms_rel = abs(v_rms_init - v_rms)/v_rms_init

if uw.mpi.rank == 0:
    print(f"step: {step}")
    print(f"Cumulative distance traveled: {dist_travel:.6f}")
    print(f"Courant number: {courant_num:.6f}")
    print(f"Omega rel norm: {om_norm:.8f}")
    print(f"V rel norm: {v_norm:.8f}")
    #print(f"Omega rel error: {om_rms_rel:.8f}")
    #print(f"Velocity rel error: {v_rms_rel:.8f}")


# mesh.write_timestep(
#         outfile,
#         meshUpdates=True,
#         meshVars=[vec_tst, vec_ana],
#         outputPath=outdir,
#         index = step,
#     )


  0 SNES Function norm 1.058890926569e-02
    Residual norms for  solve.
    0 KSP Residual norm 1.471937737751e+01
    1 KSP Residual norm 3.136125106495e+00
    2 KSP Residual norm 1.458821269347e+00
    3 KSP Residual norm 2.546302918492e-01
    4 KSP Residual norm 6.005149607875e-02
    5 KSP Residual norm 1.531720599215e-02
    6 KSP Residual norm 3.297083318282e-03
  1 SNES Function norm 1.406812550948e-06
    Residual norms for  solve.
    0 KSP Residual norm 3.303010012054e-03
    1 KSP Residual norm 1.044726535525e-03
    2 KSP Residual norm 3.097623091348e-04
    3 KSP Residual norm 1.038250944012e-04
    4 KSP Residual norm 3.540455365750e-05
    5 KSP Residual norm 1.244345921425e-05
    6 KSP Residual norm 1.840172830748e-06
  2 SNES Function norm 1.087687696179e-09
  0 SNES Function norm 1.058819423112e-02
    Residual norms for  solve.
    0 KSP Residual norm 1.471900603697e+01
    1 KSP Residual norm 3.135987862481e+00
    2 KSP Residual norm 1.458808170558e+00
    3 KS

In [16]:
mesh.get_min_radius()

0.04419417382386928

In [17]:
mesh?

Type:        Mesh
String form: <underworld3.discretisation.Mesh object at 0x167b61850>
File:        ~/Documents/codes/uw3-dev/uw3-local-installation/uw3_dev/src/underworld3/discretisation.py
Docstring:   Mesh class for uw - documentation needed